In [4]:
import inspect

# Algorithmes d'apprentissage

Au coeur de tous les algorithmes d'apprentissage présenté ici, il y a la matrice $Q_{s,a}$, qui est l'estimation relative (à l'algorithme) de chaque couple état-action. Plus la valeur du couple $(etat, action)$ est haute, plus elle est associé à un taux de réussite élevé, et pour un état donné (une ligne de la $Q_{s,a}$ ), la politique optimale associé choisira l'action ayant la valeur la plus haute.

On va ici distinguer deux familles d'algorithmes:
- d'une part la famille des algorithmes de Monte-Carlo, qui fonctionne dans un contexte épisodique, et qui a besoin d'un épisode complet mettre à jour la politique,
- d'autre part la famille des algorithme de Temporal Difference (TD), qui mette à jour al politique à chaque étape de l'épisode

## Methode(s) Monte-carlo

La méthode de **Monte-Carlo** (pour le *Reinforcement Learning*) est l'approche la plus "naïve" qu'on puisse avoir: pour évaluer la qualité d'un état, respectivement d'un couple état-action, on observe le "taux de réussite" de cet état, respectivement du couple état-action, après un épisode complet. 
 A l'issue de suffisement d'épisodes, on a une estimation relative de chaque état, ou couple état-action, à partir de laquelle on peut déduire une politique "optimale".

lien pertinent: https://towardsdatascience.com/monte-carlo-learning-b83f75233f92

**Remarques**:
- Comment faire lorsqu'on ne peut pas partir de chacun des états du système (comme c'est le cas dans *Frozen-Lake*)?
- D'une certaine manière, on "se fiche" de comment l'agent parcours les états, seul l'observation des trajectoires comptent
- Instinctivement, pour éviter de biaiser le parcours de l'agent, on peut utiliser une politique uniformément aléatoire (lors de la phase d'apprentissage)
- Avec le constat précédent, on peut simplement paramétrer un algoritme de Monte Carlo avec comme seul paramètre le nombre d'itération

In [6]:
with open("Monte_Carlo.py", 'r', encoding='utf-8') as the_file:
    print(the_file.read())

from src.V2.Classes.Policy import Policy
from src.V2.Classes.Agent import Agent

import numpy as np
import random

# Generic Initialisation
def MC(environment, epoch_number = 8000):
    # Get the observation space & the action space
    environment_space_length: int = environment.observation_space.n # type: ignore
    action_space_length: int = environment.action_space.n # type: ignore
    Q_sa = np.zeros((environment_space_length, action_space_length))
    incremental_counter = np.zeros((environment_space_length, action_space_length))
    random_policy = Policy(
        lambda agent, state: 
            random.randint(0, action_space_length - 1)
        ,
        environment_space_length,
        action_space_length
    )
    def update_MC(agent: Agent):
        # All the (state, action) pair got updated with the last reward of the run
        final_value = agent.trajectory.steps[-1].reward
        for step in agent.trajectory.steps:
            increment = incremental_counter[step.st

## Temporal Difference (TD)

Contrairement aux algoritmes "Monte Carlo", les algorithmes TD mettent à jour leur politique à chaque étape de chaque épisode.

**Remarques:**
- J'ai trouvé beaucoup de contradiction dans les définitions de *on-policy* et *off-policy*, celles que j'ai utilisée sont la version de wikipedia (et découle de l'observation des méthodes de mise à jour)
- pour ces algorithmes, on aura typiquement 4 paramètres: le nombre d'itération, le facteur d'apprentissage $\alpha$, le facteur d'actualisation $\gamma$ et $\epsilon$ le facteur d'exporation de la politique $\epsilon$*-greedy*

### SARSA

L'algorithme **SARSA** tire son nom de sa méthode d'apprentissage, qui signifie *State-Action-Reward-State-Action*: à chaque pas de l'algorithme, c'est à dire à chaque fois qu'un agent choisi une action, on va mettre à jour la matrice $Q_{s, a}$ en fonction de l'était de départ, l'action choisi par la politique, la récompense fournit par l'environnement, mais également l'action suivante à l'état suivant choisi par la politique en cours. L'amélioration de $Q_{s, a}$ peut donc s'écrire comme la fonction suivante: $update\_SARSA(State_n, Action_n, Reward_n, State_{n+1}, Action_{n+1})$.

Puisque que la fonction d'amélioration dépend de l'action $Action_{n+1}$, elle même déterminée par la politique en cours, on dit qu'il s'agit d'un algorithme *on-policy*.

In [7]:
with open("SARSA.py", 'r', encoding='utf-8') as the_file:
    print(the_file.read())

from typing import Callable
import gymnasium as gym
from nptyping import Float, NDArray, Shape
import numpy as np

from src.V2.Classes.Policy import Policy
from src.V2.Classes.Agent import Agent
from src.V2.Functions.epsilon_greedy_policy_factory import make_epsilon_greedy_policy

def SARSA(environment, epsilon = 0.1, alpha = 0.1, gamma = 0.99, epoch_number = 8000):
    # Get the observation space & the action space
    environment_space_length: int = environment.observation_space.n # type: ignore
    action_space_length: int = environment.action_space.n # type: ignore
    Q_sa = np.zeros((environment_space_length, action_space_length))

    epsilon_greedy_policy = Policy(
        make_epsilon_greedy_policy(epsilon = epsilon, Q_sa = Q_sa),
        environment_space_length,
        action_space_length
    )
    def update_SARSA(agent: Agent, state_index: int, action_index: int, next_state: int, next_action: int,  reward:float = 0.0):
        # Q[s, a] := Q[s, a] + Î±[r + Î³Q(s', a') - Q

### Q-learning

L'algorithme **Q_learning** ressemble beaucoup à l'algorithme **SARSA**: comme lui il va mettre à jour la matrice $Q_{s, a}$. L'amélioration de $Q_{s, a}$ peut s'écrire comme la fonction suivante: $update\_Qlearning(State_n, Action_n, Reward_n, State_{n+1})$.

Contrairement à la fonction d'almélioration de **SARSA**, on voit que celle de **Q_learning** ne dépend pas de l'action $Action_{n+1}$: elle ne dépend donc pas de la politique utilisé, et on parlera alors d'un algorithme *off-policy*.

In [8]:
with open("Q_learning.py", 'r', encoding='utf-8') as the_file:
    print(the_file.read())

import gymnasium as gym
import numpy as np

from src.V2.Classes.Policy import Policy
from src.V2.Classes.Agent import Agent
from src.V2.Functions.epsilon_greedy_policy_factory import make_epsilon_greedy_policy

def Q_learning(environment, epsilon = 0.1, alpha = 0.1, gamma = 0.99, epoch_number = 8000):
    # Get the observation space & the action space
    environment_space_length: int = environment.observation_space.n # type: ignore
    action_space_length: int = environment.action_space.n # type: ignore
    Q_sa = np.zeros((environment_space_length, action_space_length))

    epsilon_greedy_policy = Policy(
        make_epsilon_greedy_policy(epsilon = epsilon, Q_sa = Q_sa),
        environment_space_length,
        action_space_length
    )
    def update_Qlearning(agent: Agent, state_index, action_index, next_state, reward: float = 0):
        # Q[s, a] := Q[s, a] + α[r + γ . argmax_a {Q(s', a')} - Q(s, a)]
        best_next_action = np.argmax(Q_sa[next_state, ])
        Q_sa[state_i

# Comparaisons, convergence et métrique

Les temps de calcul des algorithmes seront générallement "long", on voudrait alors savoir quand s'arrêter avec un résultat (*i.e* une politique optimale estimée) qui converge. Avant de se lancer dans une longue période d'apprentissage, on aimerait choisir judicieusement les paramètres de notre algorithme.

**Remarques:**
- La lecture de littérature sur le sujet peut donner une bonne idée de paramètres "non-déconnants", ou d'une plage de valeur pertinente
- Ici, j'aurai une approche naïve, et j'essairai de choisir ses paramètres uniquement via l'exploitation des algorithmes précédents

## Grille

Pour essayer de me faire une idée du comportement de chaque algorithme en fonction de ses paramètre, l'approche naïve est de faire une grille sur l'espace des paramètres. Les valeurs possibles pour $\alpha$, $\gamma$ et $\epsilon$ sont toutes dans l'intervalle $[0, 1]$, et les valeurs pour le nombre d'estimation sont les entiers positifs.